In [9]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target #rotulos
df = df[['sepal length (cm)', 'petal width (cm)', 'target']] # 3 colunas
df = df[df['target'].isin([0, 1])] # seleciona calsses setosa e versicolor
# separando as clounas classes 0 e 1 e features
R = [0, 1] 
X = df.iloc[:, R]  
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LogisticRegression()
modelo.fit(X_train, y_train)
score = modelo.score(X_test, y_test)

w1 = modelo.coef_[0][0]  # Peso'sepal length'
w2 = modelo.coef_[0][1]  # Peso 'petal width'
wo = modelo.intercept_[0]  # Termo constante gerado pela reg. logistica

print(f"Pesos: {modelo.coef_}")
print(f"Intercepto: {modelo.intercept_}")
print(f"Acurácia: {score:.2f}")


Pesos: [[1.37515913 3.50507158]]
Intercepto: [-9.9889242]
Acurácia: 1.00


In [12]:
def ONEEXPLANATION(Vs, delta, R, classe):
    """
    Encontrar uma PI-explicação usando um algoritmo guloso.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        R: Limite de explicação.
        classe: Classe atual sendo considerada.

    Returns:
        Tupla com o limite atualizado (R) e o índice da feature (Idx).
    """
    idx = 0  # Inicializa o índice da feature
    while R >= 0:
        # Verifica se o índice da feature está dentro dos limites da lista delta
        if idx + 1 < len(delta):
            # Incrementa o índice da feature
            idx += 1
            # Atualiza o limite de explicação (R)
            R -= delta[idx]
            # Retorna o limite de explicação atualizado (R) e o índice da feature (idx)
            return R, idx
        else:
            # Se o índice da feature estiver fora dos limites, interrompe o loop
            return R, idx
        
delta = []
for i, valor_instancia in enumerate(a):
    if w[i] < 0:
        delta_feature = (valor_instancia - df[df.columns[i]].max()) * w[i]
    else:
        delta_feature = (valor_instancia - df[df.columns[i]].min()) * w[i]
    delta.append(delta_feature)


TypeError: ONEEXPLANATION() missing 4 required positional arguments: 'Vs', 'delta', 'R', and 'classe'

In [5]:
########## FUNÇÕES ##########
def ONEEXPLANATION(Vs, delta, R):
    """
    Encontrar uma PI-explicação usando um algoritmo guloso.
    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        R: Limite de explicação.
        Idx: Índice atual na lista delta.
        Xpl: Conjunto de literais da explicação.
        classe: Classe atual sendo considerada.
    Returns:
        Tupla com o limite atualizado (R) e o índice atualizado (Idx).
    """
    Idx = -1
    Xpl = []
    # Verifica se Idx está dentro dos limites da lista antes de incrementá-lo
    if Idx + 1 < len(delta):
        # Se o peso da primeira feature for maior
        if abs(w1) > abs(w2):
            Idx = 0 # Escolhe a primeira feature como a mais importante
        else:
            Idx = 1 # Escolhe a segunda feature como a mais importante

        R -= delta[Idx]
        # Converte a tupla para string para que seja "hashable"
        Xpl.add(str((Idx, Vs[Idx])))  # Adiciona o literal à PI-explicação
        EXPLICAR_PI(Xpl, classe, w1, w2, df)  # Imprime ou processa a PI-explicação atual
        return R, Idx
    else:
        # Se Idx estiver fora dos limites, interrompe o loop
        return R, Idx

## def ALLEXPLANATIONS(Vs, delta, threshold, w1, w2):
    """
    Enumerar todas as PI-explicações usando backtracking, considerando todas as classes.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        threshold: Limite de explicação.
        w1: Peso para a primeira feature.
        w2: Peso para a segunda feature.

    Returns:
        Lista de tuplas com a classe e a PI-explicação para cada instância.
    """
    pi_explicacoes = []  # Lista para armazenar as PI-explicações
    for classe in range(2):  # Itera sobre as duas classes (0 e 1)
        Xpl = set()  # cria um set vazio para ramazenar os literais da pi-explica
        Idx = 0
        R = 0
        
        while Idx >= 0 and Idx < len(delta): 
            R, Idx = ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe)
            # Ajuste para garantir que R não se torne negativo
            if R < 0:
                R = 0
            # Adiciona a PI-explicação à lista, apenas se não existir na lista
            if (classe, Xpl) not in pi_explicacoes:
                pi_explicacoes.append((classe, Xpl))  # Adiciona a PI-explicação à lista
            Idx += 1 # Incrementa Idx após a chamada da função ONEEXPLANATION
    return pi_explicacoes
# print(pi_explicacoes)
# print(delta)
def EXPLICAR_PI(Xpl, classe, w1, w2, df):
    """Imprime a PI-explicação."""
    print(f"Classe: {classe}")
    print(f"PI-explicação: {Xpl}")
    for item in Xpl:
         idx, valores = eval(item)
         if idx == 0:
             print(f"  - Sepal Length ({w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width ({w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")
         else:
             print(f"  - Sepal Length (peso - {w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width (peso - {w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")

####### delta_feature = (valor_instancia - df[feature].max()) * w_feature, se #w_feature < 0
#     delta_feature = (valor_instancia - df[feature].min()) * w_feature, se #w_feature > 0
#assumindo predicao para a classe +1

'''delta_feature = (valor_instancia - df[feature].max()) * w_feature, se w_feature < 0
     delta_feature = (valor_instancia - df[feature].min()) * w_feature, se w_feature > 0
assumindo predicao para a classe +1'''
delta = []
for feature in df.columns[:-1]:  # Exclui a ultima coluna 'target'
    delta_feature = df[feature].max() - df[feature].min()
    delta.append(delta_feature)

# ordenar a saida do delta "sem perder os valores das features que eles se referem"

# com o(threshold) mais baixo o classificador pode ficar mais rigoroso
threshold = 0 

Vs = []
for index, row in df.iterrows():
    Vs.append(list(row[:-1]))  # adiciona os pares Sepal e petal

print(Vs)

pi_explicacoes = ALLEXPLANATIONS(Vs, delta, threshold, w1, w2)

# Cria um DataFrame com as PI-explicações
pi_explicacoes_df = pd.DataFrame(pi_explicacoes, columns=['Classe', 'PI-Explicação'])

# print(f'\n{pi_explicacoes_df}')

[[5.1, 0.2], [4.9, 0.2], [4.7, 0.2], [4.6, 0.2], [5.0, 0.2], [5.4, 0.4], [4.6, 0.3], [5.0, 0.2], [4.4, 0.2], [4.9, 0.1], [5.4, 0.2], [4.8, 0.2], [4.8, 0.1], [4.3, 0.1], [5.8, 0.2], [5.7, 0.4], [5.4, 0.4], [5.1, 0.3], [5.7, 0.3], [5.1, 0.3], [5.4, 0.2], [5.1, 0.4], [4.6, 0.2], [5.1, 0.5], [4.8, 0.2], [5.0, 0.2], [5.0, 0.4], [5.2, 0.2], [5.2, 0.2], [4.7, 0.2], [4.8, 0.2], [5.4, 0.4], [5.2, 0.1], [5.5, 0.2], [4.9, 0.2], [5.0, 0.2], [5.5, 0.2], [4.9, 0.1], [4.4, 0.2], [5.1, 0.2], [5.0, 0.3], [4.5, 0.3], [4.4, 0.2], [5.0, 0.6], [5.1, 0.4], [4.8, 0.3], [5.1, 0.2], [4.6, 0.2], [5.3, 0.2], [5.0, 0.2], [7.0, 1.4], [6.4, 1.5], [6.9, 1.5], [5.5, 1.3], [6.5, 1.5], [5.7, 1.3], [6.3, 1.6], [4.9, 1.0], [6.6, 1.3], [5.2, 1.4], [5.0, 1.0], [5.9, 1.5], [6.0, 1.0], [6.1, 1.4], [5.6, 1.3], [6.7, 1.4], [5.6, 1.5], [5.8, 1.0], [6.2, 1.5], [5.6, 1.1], [5.9, 1.8], [6.1, 1.3], [6.3, 1.5], [6.1, 1.2], [6.4, 1.3], [6.6, 1.4], [6.8, 1.4], [6.7, 1.7], [6.0, 1.5], [5.7, 1.0], [5.5, 1.1], [5.5, 1.0], [5.8, 1.2], [6.